In [103]:
import os

import pandas as pd
import numpy as np

import scipy.stats as stats
from scipy.stats import ttest_ind

from datetime import datetime, timedelta

from tqdm.notebook import tqdm

In [104]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Задание 1

С одной стороны, при удалении выбросов снижается дисперсия, что приводит к увеличению чувствительности теста. С другой стороны, при удалении выбросов уменьшается размер выборки, что приводит к уменьшению чувствительности.

Сравните мощности тестов с разной долей удаляемых данных. Используйте данные о времени работы бэкенда 2022-04-01T12_df_web_logs.csv в период с 2022-03-01 по 2022-03-08. Уровень значимости — 0.05. Размеры групп — 1000 человек (размер выборок будет больше, так как на одного человека приходится много значений). Проверяем гипотезу о равенстве средних с помощью теста Стьюдента. Ожидаемый эффект — увеличение времени обработки на 1%. Эффект в синтетических А/В-тестах добавляем умножением на константу.

В ответ введите номера вариантов, упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между np.quantile(values, 0.01) и np.quantile(values, 0.99). Квантили вычислять для каждой групп отдельно.

In [105]:
web_logs = read_database('2022-04-01T12_df_web_logs.csv')
web_logs['date'] = pd.to_datetime(web_logs['date'])
web_logs.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [106]:
web_logs_hist = web_logs[
    (web_logs['date'] >= datetime(2022, 3, 1))
    & (web_logs['date'] < datetime(2022, 3, 8))
].reset_index(drop=True)
web_logs_hist.shape

(245851, 4)

In [108]:
crops = [0.0002, 0.002, 0.02, 0.1, 0.2]
effect = 0.01
crops_error = []
alpha = 0.05

user_ids = web_logs_hist['user_id'].values

for crop in crops:
    p_values = []

    for i in range(1000):
        np.random.shuffle(user_ids)
        load_time_a = web_logs_hist.loc[web_logs_hist['user_id'].isin(user_ids[:1000]), 'load_time'].values
        load_time_b = web_logs_hist.loc[web_logs_hist['user_id'].isin(user_ids[1000:2000]), 'load_time'].values
        load_time_b *= (1 + effect)
        
        lower_quantile_a = np.quantile(load_time_a, crop / 2)
        upper_quantile_a = np.quantile(load_time_a, 1 - crop / 2)
        load_time_a = load_time_a[(load_time_a > lower_quantile_a) & (load_time_a < upper_quantile_a)]
        
        lower_quantile_b = np.quantile(load_time_b, crop / 2)
        upper_quantile_b = np.quantile(load_time_b, 1 - crop / 2)
        load_time_b = load_time_b[(load_time_b > lower_quantile_b) & (load_time_b < upper_quantile_b)]
        
        _, p_val = ttest_ind(load_time_a, load_time_b)
        p_values.append(p_val)

    crops_error.append((np.array(p_values) < alpha).mean())

for crop, error in zip(crops, crops_error):
    print(f"Crop level: {crop}, Power rate: {error}")

Crop level: 0.0002, Power rate: 0.106
Crop level: 0.002, Power rate: 0.415
Crop level: 0.02, Power rate: 0.993
Crop level: 0.1, Power rate: 1.0
Crop level: 0.2, Power rate: 0.995


# Задание 2

Выполните то же задание, изменив способ добавления эффекта. Эффект в синтетических А/В-тестах добавляем добавлением константы к 1% данных.

В ответ введите номера вариантов упорядоченные по уменьшению мощности. Например, «12345» означает, что вариант 1 обладает наибольшей мощностью, а вариант 5 — наименьшей.

1. Удалить 0.02% выбросов;

2. Удалить 0.2% выбросов;

3. Удалить 2% выбросов;

4. Удалить 10% выбросов;

5. Удалить 20% выбросов.

Удалить 2% выбросов означает, что нужно убрать по 1% минимальных и максимальных значений выборки. То есть оставить значения, которые лежат между np.quantile(values, 0.01) и np.quantile(values, 0.99). Квантили вычислять для каждой группы отдельно.

In [107]:
crops = [0.0002, 0.002, 0.02, 0.1, 0.2]
effect = 0.01
crops_error = []
alpha = 0.05

user_ids = web_logs_hist['user_id'].values

for crop in crops:
    p_values = []

    for i in range(1000):
        np.random.shuffle(user_ids)

        load_time_a = web_logs_hist.loc[web_logs_hist['user_id'].isin(user_ids[:1000]), 'load_time'].values
        load_time_b = web_logs_hist.loc[web_logs_hist['user_id'].isin(user_ids[1000:2000]), 'load_time'].values

        mean_ = load_time_b.mean()
        sample_size = len(load_time_b)
        indexes = np.random.choice(np.arange(sample_size), int(sample_size * 0.01), False)
        add_value = effect * mean_ * sample_size / len(indexes)
        mask = np.zeros(sample_size)
        mask[indexes] += 1
        load_time_b = load_time_b + mask * add_value
        
        lower_quantile_a = np.quantile(load_time_a, crop / 2)
        upper_quantile_a = np.quantile(load_time_a, 1 - crop / 2)
        load_time_a = load_time_a[(load_time_a > lower_quantile_a) & (load_time_a < upper_quantile_a)]
        
        lower_quantile_b = np.quantile(load_time_b, crop / 2)
        upper_quantile_b = np.quantile(load_time_b, 1 - crop / 2)
        load_time_b = load_time_b[(load_time_b > lower_quantile_b) & (load_time_b < upper_quantile_b)]
        
        _, p_val = ttest_ind(load_time_a, load_time_b)
        p_values.append(p_val)
    crops_error.append((np.array(p_values) < alpha).mean())

for crop, error in zip(crops, crops_error):
    print(f"Crop level: {crop}, Power rate: {error}")

Crop level: 0.0002, Power rate: 0.118
Crop level: 0.002, Power rate: 0.465
Crop level: 0.02, Power rate: 0.735
Crop level: 0.1, Power rate: 0.545
Crop level: 0.2, Power rate: 0.536


In [110]:
sample_size = 1000
effect = 0.01
alpha = 0.05
quantiles = (0.0001, 0.001, 0.01, 0.05, 0.1,)
quantile2errors = {q: [] for q in quantiles}

for _ in range(10000):
    a_users, b_users = np.random.choice(user_ids, (2, sample_size,), False)
    a_values = web_logs_hist.loc[web_logs_hist['user_id'].isin(a_users), 'load_time'].values
    b_values = web_logs_hist.loc[web_logs_hist['user_id'].isin(b_users), 'load_time'].values
    mean_ = b_values.mean()
    indexes = np.random.choice(np.arange(len(b_values)), int(len(b_values) / 100), False)
    b_values[indexes] += mean_ * effect * len(b_values) / len(indexes)

    for q in quantiles:
        a_values_filtered = a_values[
            (a_values > np.quantile(a_values, q))
            & (a_values < np.quantile(a_values, 1-q))
        ]
        b_values_filtered = b_values[
            (b_values > np.quantile(b_values, q))
            & (b_values < np.quantile(b_values, 1-q))
        ]
        pvalue = stats.ttest_ind(a_values_filtered, b_values_filtered).pvalue
        quantile2errors[q].append(pvalue > alpha)

quantile2errors


{0.0001: [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  